In [1]:
import numpy as np
import pandas as pd


In [2]:
dataset = pd.read_csv('Case_cobranca.csv') 

In [3]:
#------------------------------------------------------------------------------------------
# Pré-processamento das variáveis
#------------------------------------------------------------------------------------------
## Tratamento de nulos no alvo --- Tempo de Atraso - transformação para alvo binário (>90 dias) 
dataset['ALVO']   = [0 if np.isnan(x) or x > 90 else 1 for x in dataset['TEMP_RECUPERACAO']]
## Tratamento de nulos e normalização --- Variáveis de entrada numéricas
dataset['PRE_IDADE']        = [18 if np.isnan(x) or x < 18 else x for x in dataset['IDADE']] # Trata mínimo
dataset['PRE_IDADE']        = [1. if x > 76 else (x-18)/(76-18) for x in dataset['PRE_IDADE']] # Trata máximo por percentil 99 e coloca na fórmula
dataset['PRE_QTDE_DIVIDAS'] = [0.  if np.isnan(x) else x/16. for x in dataset['QTD_DIVIDAS']] # retirada de outlier com percentil 99 e normalização     
##--- Dummies - transformação de atributos categóricos em numéricos e tratamanto de nulos ---------------
dataset['PRE_NOVO']         = [1 if x=='NOVO'                      else 0 for x in dataset['TIPO_CLIENTE']]    
dataset['PRE_TOMADOR_VAZIO']= [1 if x=='TOMADOR' or str(x)=='nan'  else 0 for x in dataset['TIPO_CLIENTE']]                        
dataset['PRE_CDC']          = [1 if x=='CDC'                       else 0 for x in dataset['TIPO_EMPRESTIMO']]
dataset['PRE_PESSOAL']      = [1 if x=='PESSOAL'                   else 0 for x in dataset['TIPO_EMPRESTIMO']]
dataset['PRE_SEXO_M']       = [1 if x=='M'                         else 0 for x in dataset['CD_SEXO']]


In [5]:
##------------------------------------------------------------
## Separando em dados de treinamento e teste
##------------------------------------------------------------
y = dataset['ALVO']              # Carrega alvo ou dataset.iloc[:,7].values
X = dataset.iloc[:, 8:15].values # Carrega colunas 8, 9, 10, 11, 12, 13 e 14 (a 15 não existe até este momento)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 25)


In [6]:
#---------------------------------------------------------------------------
## Ajustando - Aprendizado supervisionado  
#---------------------------------------------------------------------------
# Regressão logística com dados de treinamento
from sklearn.linear_model import LogisticRegression
LogisticReg = LogisticRegression()
LogisticReg.fit(X_train, y_train)

LogisticRegression()

In [7]:
#---------------------------------------------------------------------------
## Calculando a KNN - Aprendizado supervisionado  
#---------------------------------------------------------------------------
from sklearn.neighbors import KNeighborsClassifier
Classifier_kNN = KNeighborsClassifier(n_neighbors=30, weights='uniform', algorithm='brute', p=2)
Classifier_kNN.fit(X_train, y_train)

KNeighborsClassifier(algorithm='brute', n_neighbors=30)

In [8]:
# Árvore de decisão com dados de treinamento
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0,
                       min_samples_leaf=100, min_samples_split=200,
                       min_weight_fraction_leaf=0.0,
                       random_state=0, splitter='best')
dtree.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', min_samples_leaf=100,
                       min_samples_split=200, random_state=0)

In [9]:
#---------------------------------------------------------------------------
## Previsão usando todos os conjuntos de teste
#---------------------------------------------------------------------------
# Rede Neural
y_pred_test_RLog  = LogisticReg.predict(X_test)
y_pred_test_RNA_P  = LogisticReg.predict_proba(X_test)


In [10]:
# Knn
y_pred_test_KNN    = Classifier_kNN.predict(X_test)
#y_pred_test_KNN_P  = Classifier_kNN.predict_proba(X_test)

c:\Users\rafaela.a.dos.santos\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\rafaela.a.dos.santos\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


In [11]:
#  Árvore
y_pred_test_DT = dtree.predict(X_test)

In [12]:
##-----------------------------------------------------------------
## Cálculo dos erros da classificação e Matriz de confusão da RNA
##-----------------------------------------------------------------
Erro_RLog_Classificacao = np.mean(np.absolute(y_pred_test_RLog - y_test))
Erro_KNN_Classificacao = np.mean(np.absolute(y_pred_test_KNN - y_test))
Erro_DT_Classificacao = np.mean(np.absolute(y_pred_test_DT - y_test))

print()
print('---------------------------------------------')
print('Regressão Logística  - Erro de Classificação:',Erro_RLog_Classificacao)
print('Regressão Logística  - Acurácia:',1-Erro_RLog_Classificacao)
print('KNN                  - Erro de Classificação:',Erro_KNN_Classificacao)
print('KNN                  - Acurácia:',1-Erro_KNN_Classificacao)
print('Árvore               - Erro de Classificação:',Erro_DT_Classificacao)
print('Árvore               - Acurácia:',1-Erro_DT_Classificacao)
print('----------------------------------------------')


---------------------------------------------
Regressão Logística  - Erro de Classificação: 0.2704684317718941
Regressão Logística  - Acurácia: 0.7295315682281058
KNN                  - Erro de Classificação: 0.2659877800407332
KNN                  - Acurácia: 0.7340122199592668
Árvore               - Erro de Classificação: 0.2602851323828921
Árvore               - Acurácia: 0.7397148676171079
----------------------------------------------
